In [1]:
import pandas as pd
import os

# ==========================================
# Step 1: 文件源锁定与基础质检 (Source Locking)
# ==========================================

# 1. 配置绝对路径
base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"

# 2. 定义目标文件 (只锁定 Drive 数据，剔除 Noise)
node_file = 'nodes_drive.csv'
edge_file = 'edges_drive.csv'

print(f"=== 开始执行步骤 1: 数据源锁定 ===")
print(f"数据路径: {base_path}")

try:
    # 3. 加载数据
    nodes_path = os.path.join(base_path, node_file)
    edges_path = os.path.join(base_path, edge_file)
    
    nodes_df = pd.read_csv(nodes_path)
    edges_df = pd.read_csv(edges_path)
    
    print(f"\n[成功加载] 节点表: {len(nodes_df)} 行")
    print(f"[成功加载] 边表:   {len(edges_df)} 行")

    # ==========================================
    # 4. 关键点校验 (Critical Validations)
    # ==========================================
    
    # Check A: 节点 ID 唯一性 (防止图构建时覆盖节点)
    if nodes_df['osmid'].is_unique:
        print(f"✅ [校验通过] 节点 ID (osmid) 唯一。")
    else:
        dup_count = nodes_df['osmid'].duplicated().sum()
        print(f"❌ [警告] 发现 {dup_count} 个重复节点 ID！请检查数据源。")

    # Check B: 坐标完整性 (后续 KDTree 匹配必须)
    missing_coords = nodes_df[['x', 'y']].isnull().sum().sum()
    if missing_coords == 0:
        print(f"✅ [校验通过] 所有节点均包含完整经纬度坐标 (x, y)。")
    else:
        print(f"❌ [致命错误] 有 {missing_coords} 个节点缺失坐标，模型将无法建立空间索引。")

    # Check C: 拓扑完整性 (u, v 必须存在)
    missing_topo = edges_df[['u', 'v', 'length']].isnull().sum().sum()
    if missing_topo == 0:
        print(f"✅ [校验通过] 所有边均包含起点(u)、终点(v)和长度(length)。")
    else:
        print(f"❌ [致命错误] 边表缺失拓扑信息，路网已断裂。")

    # Check D: 预警脏数据 (为下一步清洗做准备)
    # 检查 maxspeed 是否包含非数字字符（如 'mph'）
    if edges_df['maxspeed'].dtype == 'object':
        example_dirty = edges_df[edges_df['maxspeed'].astype(str).str.contains('mph', na=False)]['maxspeed'].iloc[0]
        print(f"⚠️ [注意] 发现 'maxspeed' 包含字符 (例如: '{example_dirty}')。")
        print(f"   -> 这将在下一步【字段清洗】中处理。")
    
    print("\n=== 步骤 1 完成: 数据源已锁定且基础质量合格 ===")
    
except FileNotFoundError as e:
    print(f"\n❌ [错误] 找不到文件，请检查路径是否正确。\n详细信息: {e}")
except Exception as e:
    print(f"\n❌ [错误] 发生未预期的错误: {e}")

=== 开始执行步骤 1: 数据源锁定 ===
数据路径: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data

[成功加载] 节点表: 37163 行
[成功加载] 边表:   91227 行
❌ [警告] 发现 1 个重复节点 ID！请检查数据源。
✅ [校验通过] 所有节点均包含完整经纬度坐标 (x, y)。
✅ [校验通过] 所有边均包含起点(u)、终点(v)和长度(length)。
⚠️ [注意] 发现 'maxspeed' 包含字符 (例如: '55 mph')。
   -> 这将在下一步【字段清洗】中处理。

=== 步骤 1 完成: 数据源已锁定且基础质量合格 ===


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\4193017283.py:24: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(edges_path)


In [2]:
import pandas as pd
import os

# 1. 配置路径
base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"

print("=== 正在读取数据 ===")

# 2. 读取数据 (加入 low_memory=False 解决 DtypeWarning 报错)
nodes_df = pd.read_csv(os.path.join(base_path, 'nodes_drive.csv'))
edges_df = pd.read_csv(os.path.join(base_path, 'edges_drive.csv'), low_memory=False)

print("✅ 数据读取完成，混合类型警告已解决。")

# 3. 找出所有重复的 ID
# keep=False 表示把所有重复的条目都列出来，方便对比
dup_mask = nodes_df.duplicated(subset=['osmid'], keep=False)
duplicate_rows = nodes_df[dup_mask]

print(f"\n=== 重复 ID 检查结果 ===")
if not duplicate_rows.empty:
    print(f"发现 {len(duplicate_rows)} 行涉及重复 ID。具体内容如下：")
    # 打印出来供你决策
    print(duplicate_rows)
else:
    print("未发现重复的 osmid。")

=== 正在读取数据 ===
✅ 数据读取完成，混合类型警告已解决。

=== 重复 ID 检查结果 ===
发现 2 行涉及重复 ID。具体内容如下：
          osmid          y          x          highway  ref  street_count  \
15489  37802386  39.371929 -76.711298  traffic_signals  NaN             4   
24574  37802386  39.371929 -76.711298  traffic_signals  NaN             4   

      junction railway                       geometry  
15489      NaN     NaN  POINT (-76.7112977 39.371929)  
24574      NaN     NaN  POINT (-76.7112977 39.371929)  


In [7]:
import pandas as pd
import os

# 1. 这里补上路径（必须有这一步）
base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
file_path = os.path.join(base_path, 'nodes_drive.csv') 

# 2. 这里的变量名要是 nodes_df (对应您 Cell 2 读出来的名字)
if 24574 in nodes_df.index:
    nodes_df.drop(index=24574, inplace=True) # 这里改 nodes_df
    print("✅ 已成功从内存中剔除索引 24574。")
else:
    print("⚠️ 索引 24574 不存在（可能已被剔除）。")

try:
    # 3. 这里保存也要用 nodes_df
    nodes_df.to_csv(file_path, index=False) 
    print(f"💾 数据已成功覆盖保存至: {file_path}")
    
except Exception as e:
    print(f"❌ 保存失败: {e}")

⚠️ 索引 24574 不存在（可能已被剔除）。
💾 数据已成功覆盖保存至: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\nodes_drive.csv


In [8]:
import pandas as pd
import os

# ==========================================
# Step 1: 文件源锁定与基础质检 (Source Locking)
# ==========================================

# 1. 配置绝对路径
base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"

# 2. 定义目标文件 (只锁定 Drive 数据，剔除 Noise)
node_file = 'nodes_drive.csv'
edge_file = 'edges_drive.csv'

print(f"=== 开始执行步骤 1: 数据源锁定 ===")
print(f"数据路径: {base_path}")

try:
    # 3. 加载数据
    nodes_path = os.path.join(base_path, node_file)
    edges_path = os.path.join(base_path, edge_file)
    
    nodes_df = pd.read_csv(nodes_path)
    edges_df = pd.read_csv(edges_path)
    
    print(f"\n[成功加载] 节点表: {len(nodes_df)} 行")
    print(f"[成功加载] 边表:   {len(edges_df)} 行")

    # ==========================================
    # 4. 关键点校验 (Critical Validations)
    # ==========================================
    
    # Check A: 节点 ID 唯一性 (防止图构建时覆盖节点)
    if nodes_df['osmid'].is_unique:
        print(f"✅ [校验通过] 节点 ID (osmid) 唯一。")
    else:
        dup_count = nodes_df['osmid'].duplicated().sum()
        print(f"❌ [警告] 发现 {dup_count} 个重复节点 ID！请检查数据源。")

    # Check B: 坐标完整性 (后续 KDTree 匹配必须)
    missing_coords = nodes_df[['x', 'y']].isnull().sum().sum()
    if missing_coords == 0:
        print(f"✅ [校验通过] 所有节点均包含完整经纬度坐标 (x, y)。")
    else:
        print(f"❌ [致命错误] 有 {missing_coords} 个节点缺失坐标，模型将无法建立空间索引。")

    # Check C: 拓扑完整性 (u, v 必须存在)
    missing_topo = edges_df[['u', 'v', 'length']].isnull().sum().sum()
    if missing_topo == 0:
        print(f"✅ [校验通过] 所有边均包含起点(u)、终点(v)和长度(length)。")
    else:
        print(f"❌ [致命错误] 边表缺失拓扑信息，路网已断裂。")

    # Check D: 预警脏数据 (为下一步清洗做准备)
    # 检查 maxspeed 是否包含非数字字符（如 'mph'）
    if edges_df['maxspeed'].dtype == 'object':
        example_dirty = edges_df[edges_df['maxspeed'].astype(str).str.contains('mph', na=False)]['maxspeed'].iloc[0]
        print(f"⚠️ [注意] 发现 'maxspeed' 包含字符 (例如: '{example_dirty}')。")
        print(f"   -> 这将在下一步【字段清洗】中处理。")
    
    print("\n=== 步骤 1 完成: 数据源已锁定且基础质量合格 ===")
    
except FileNotFoundError as e:
    print(f"\n❌ [错误] 找不到文件，请检查路径是否正确。\n详细信息: {e}")
except Exception as e:
    print(f"\n❌ [错误] 发生未预期的错误: {e}")

=== 开始执行步骤 1: 数据源锁定 ===
数据路径: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data

[成功加载] 节点表: 37162 行
[成功加载] 边表:   91227 行
✅ [校验通过] 节点 ID (osmid) 唯一。
✅ [校验通过] 所有节点均包含完整经纬度坐标 (x, y)。
✅ [校验通过] 所有边均包含起点(u)、终点(v)和长度(length)。
⚠️ [注意] 发现 'maxspeed' 包含字符 (例如: '55 mph')。
   -> 这将在下一步【字段清洗】中处理。

=== 步骤 1 完成: 数据源已锁定且基础质量合格 ===


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\4193017283.py:24: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(edges_path)


In [9]:
import pandas as pd
import ast
import re
import numpy as np
import os

# ================= 配置路径 =================
# 设置工作目录为指定的绝对路径
WORK_DIR = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"

# 输入文件 (假设源文件也在该目录下，如果不是请修改此处)
INPUT_FILE = os.path.join(WORK_DIR, "edges_drive.csv")
# 输出文件
OUTPUT_FILE = os.path.join(WORK_DIR, "edges_drive_step2.csv")

# ================= 清洗函数定义 =================

def clean_maxspeed_v2(val):
    """
    清洗 maxspeed 字段:
    1. 去除 mph 单位
    2. 处理列表字符串 ['45', '60'] -> 取平均值
    3. 统一转换为 m/s (乘以 0.44704)
    """
    if pd.isna(val): return np.nan
    
    # 辅助内部函数：提取数字
    def get_num(s):
        match = re.search(r"([0-9\.]+)", str(s))
        return float(match.group(1)) if match else None

    # 逻辑：如果是列表字符串，解析并取平均；否则直接提取
    val_num = np.nan
    if isinstance(val, str) and val.strip().startswith('['):
        try:
            val_list = ast.literal_eval(val)
            nums = [get_num(v) for v in val_list if get_num(v) is not None]
            val_num = np.mean(nums) if nums else np.nan
        except: 
            val_num = np.nan
    else:
        val_num = get_num(val)
        
    # 换算 mph -> m/s (1 mph = 0.44704 m/s)
    return val_num * 0.44704 if pd.notna(val_num) else np.nan

def clean_lanes_v2(val):
    """
    清洗 lanes 字段:
    1. 处理列表字符串 ['2', '3'] -> 取平均值 (如 2.5)
    2. 保留为浮点数，用于后续容量计算
    """
    if pd.isna(val): return np.nan
    
    def get_num(s):
        match = re.search(r"([0-9\.]+)", str(s))
        return float(match.group(1)) if match else None

    if isinstance(val, str) and val.strip().startswith('['):
        try:
            val_list = ast.literal_eval(val)
            nums = [get_num(v) for v in val_list if get_num(v) is not None]
            return np.mean(nums) if nums else np.nan
        except: 
            return np.nan
    else:
        return get_num(val)

# ================= 主执行逻辑 =================

if __name__ == "__main__":
    print(f"正在读取文件: {INPUT_FILE} ...")
    
    # 1. 读取数据
    try:
        edges_df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print(f"错误: 找不到文件 {INPUT_FILE}，请检查路径。")
        exit()

    # 2. 应用清洗 (Type Cleaning)
    print("正在清洗 'maxspeed' (转换为 m/s)...")
    edges_df['maxspeed'] = edges_df['maxspeed'].apply(clean_maxspeed_v2)

    print("正在清洗 'lanes' (处理列表并保留浮点数)...")
    edges_df['lanes'] = edges_df['lanes'].apply(clean_lanes_v2)

    # 3. 检查结果预览
    print("\n清洗后数据预览 (Head 5):")
    print(edges_df[['maxspeed', 'lanes']].head())
    
    print("\n清洗统计:")
    print(f"Maxspeed 非空数量: {edges_df['maxspeed'].notna().sum()}")
    print(f"Lanes 非空数量:    {edges_df['lanes'].notna().sum()}")

    # 4. 保存文件
    print(f"\n正在保存文件至: {OUTPUT_FILE} ...")
    edges_df.to_csv(OUTPUT_FILE, index=False)
    print("✅ 步骤 2 完成: 字段类型清洗已保存。")

正在读取文件: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive.csv ...


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\1465836527.py:76: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(INPUT_FILE)


正在清洗 'maxspeed' (转换为 m/s)...
正在清洗 'lanes' (处理列表并保留浮点数)...

清洗后数据预览 (Head 5):
   maxspeed  lanes
0       NaN    NaN
1       NaN    NaN
2       NaN    NaN
3       NaN    NaN
4       NaN    NaN

清洗统计:
Maxspeed 非空数量: 48971
Lanes 非空数量:    44467

正在保存文件至: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step2.csv ...
✅ 步骤 2 完成: 字段类型清洗已保存。


In [10]:
import pandas as pd
import numpy as np
import ast
import os

# ================= 配置路径 =================
# 输入文件 (Step 2 的输出)
INPUT_FILE = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step2.csv"
# 输出文件 (Step 3 的结果)
OUTPUT_FILE = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step3.csv"

# ================= 1. 定义分级填补字典 (Hierarchical Dictionary) =================
# 单位说明: 
# Speed: m/s (1 mph = 0.44704 m/s)
# Lanes: float

imputation_dict = {
    # --- 高速公路系统 (Motorways) ---
    'motorway':      {'speed': 29.06, 'lanes': 3.0}, # ~65 mph
    'motorway_link': {'speed': 13.41, 'lanes': 1.0}, # ~30 mph (匝道单车道)
    
    # --- 快速路系统 (Trunk) ---
    'trunk':         {'speed': 20.12, 'lanes': 3.0}, # ~45 mph
    'trunk_link':    {'speed': 13.41, 'lanes': 1.0}, # ~30 mph
    
    # --- 主干道系统 (Primary) ---
    'primary':       {'speed': 13.41, 'lanes': 2.0}, # ~30 mph
    'primary_link':  {'speed': 11.18, 'lanes': 1.0}, # ~25 mph
    
    # --- 次干道系统 (Secondary) ---
    'secondary':     {'speed': 13.41, 'lanes': 2.0}, # ~30 mph
    'secondary_link':{'speed': 11.18, 'lanes': 1.0}, # ~25 mph
    
    # --- 支路系统 (Tertiary) ---
    'tertiary':      {'speed': 11.18, 'lanes': 2.0}, # ~25 mph
    'tertiary_link': {'speed': 11.18, 'lanes': 1.0}, # ~25 mph
    
    # --- 居住区与低速道路 ---
    'residential':   {'speed': 8.94,  'lanes': 1.0}, # ~20 mph (保守策略:防止过境穿插)
    'living_street': {'speed': 6.71,  'lanes': 1.0}, # ~15 mph
    'unclassified':  {'speed': 8.94,  'lanes': 1.0}, # ~20 mph
    'service':       {'speed': 6.71,  'lanes': 1.0}  # ~15 mph (服务道路)
}

# --- 全局兜底策略 (Fallback) ---
# 当 highway 类型不在字典中时使用 (如 'busway', 'path' 等)
GLOBAL_DEFAULT = {'speed': 8.94, 'lanes': 1.0} # ~20 mph, 1 lane

# ================= 2. 核心填补函数 =================

def fill_missing_values(row):
    """
    对每一行应用分级填补逻辑
    """
    # --- A. 获取并清洗 highway 类型 ---
    hw_raw = row['highway']
    
    # 如果是列表字符串 "['primary', 'residential']"，取第一个
    if isinstance(hw_raw, str) and hw_raw.strip().startswith('['):
        try:
            hw_list = ast.literal_eval(hw_raw)
            hw_type = hw_list[0] if hw_list else 'unclassified'
        except:
            hw_type = 'unclassified'
    else:
        hw_type = str(hw_raw)
        
    # --- B. 查字典获取默认值 ---
    defaults = imputation_dict.get(hw_type, GLOBAL_DEFAULT)
    
    # --- C. 填补 maxspeed ---
    speed = row['maxspeed']
    if pd.isna(speed):
        speed = defaults['speed']
        
    # --- D. 填补 lanes ---
    lanes = row['lanes']
    if pd.isna(lanes):
        lanes = defaults['lanes']
        
    return speed, lanes

# ================= 3. 主执行逻辑 =================

if __name__ == "__main__":
    print(f"正在读取文件: {INPUT_FILE} ...")
    
    if not os.path.exists(INPUT_FILE):
        print(f"❌ 错误: 找不到文件 {INPUT_FILE}，请检查路径。")
    else:
        # 读取数据
        edges_df = pd.read_csv(INPUT_FILE)
        
        # 统计填补前的缺失情况
        missing_speed_before = edges_df['maxspeed'].isna().sum()
        missing_lanes_before = edges_df['lanes'].isna().sum()
        print(f"填补前缺失值 -> Maxspeed: {missing_speed_before}, Lanes: {missing_lanes_before}")
        
        # 执行填补
        print("正在应用分级填补策略 (Hierarchical Imputation)...")
        # 使用 apply 获取填补后的两列数据
        filled_data = edges_df.apply(fill_missing_values, axis=1, result_type='expand')
        
        # 将结果赋值回原 DataFrame
        edges_df['maxspeed'] = filled_data[0]
        edges_df['lanes'] = filled_data[1]
        
        # 统计填补后的缺失情况 (理论上应为 0)
        missing_speed_after = edges_df['maxspeed'].isna().sum()
        missing_lanes_after = edges_df['lanes'].isna().sum()
        print(f"填补后缺失值 -> Maxspeed: {missing_speed_after}, Lanes: {missing_lanes_after}")
        
        # 如果仍有 NaN (极少见情况), 强制使用全局兜底
        if missing_speed_after > 0:
            print("⚠️ 警告: 仍有 maxspeed 缺失，强制应用全局兜底...")
            edges_df['maxspeed'] = edges_df['maxspeed'].fillna(GLOBAL_DEFAULT['speed'])
            
        if missing_lanes_after > 0:
            print("⚠️ 警告: 仍有 lanes 缺失，强制应用全局兜底...")
            edges_df['lanes'] = edges_df['lanes'].fillna(GLOBAL_DEFAULT['lanes'])

        # 保存文件
        print(f"正在保存文件至: {OUTPUT_FILE} ...")
        edges_df.to_csv(OUTPUT_FILE, index=False)
        print("✅ 步骤 3 完成: 缺失值分级填补已保存。")

正在读取文件: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step2.csv ...


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\3980590184.py:92: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv(INPUT_FILE)


填补前缺失值 -> Maxspeed: 42256, Lanes: 46760
正在应用分级填补策略 (Hierarchical Imputation)...
填补后缺失值 -> Maxspeed: 0, Lanes: 0
正在保存文件至: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step3.csv ...
✅ 步骤 3 完成: 缺失值分级填补已保存。


In [11]:
import pandas as pd

# 设置文件的绝对路径
file_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step3.csv"

try:
    # 读取CSV文件
    # 由于不确定文件编码，默认尝试utf-8，如果报错可以尝试 encoding='gbk'
    df = pd.read_csv(file_path)
    
    # 检查是否存在 'oneway' 列
    if 'oneway' in df.columns:
        # 获取 'oneway' 列的所有唯一值
        unique_values = df['oneway'].unique()
        
        print(f"文件读取成功。")
        print(f"数据总行数: {len(df)}")
        print("-" * 30)
        print("oneway列中出现的所有唯一内容如下：")
        print("-" * 30)
        
        # 逐个打印唯一值，方便查看
        for val in unique_values:
            print(val)
            # 如果想看该值的具体类型（是字符串还是布尔值），可以取消下面注释
            # print(f"值: {val}, 类型: {type(val)}")
            
    else:
        print("错误：数据集中未找到名为 'oneway' 的列，请检查列名。")

except FileNotFoundError:
    print(f"错误：找不到文件，请确认路径是否正确：\n{file_path}")
except Exception as e:
    print(f"读取文件时发生错误：{e}")

文件读取成功。
数据总行数: 91227
------------------------------
oneway列中出现的所有唯一内容如下：
------------------------------
False
True


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\3381970734.py:9: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [18]:
import pandas as pd
import shapely.wkt
from shapely.geometry import LineString
import os

# ================= 配置区域：绝对路径 =================
# 输入文件路径
input_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step3.csv"
# 输出文件路径 (保存在同一目录下)
output_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step4.csv"
# ===================================================

def process_bidirectional_edges():
    print(f"正在读取文件: {input_file} ...")
    
    # 检查文件是否存在
    if not os.path.exists(input_file):
        print(f"❌ 错误: 找不到文件，请检查路径是否正确：\n{input_file}")
        return

    df = pd.read_csv(input_file)

    # 1. 清洗 oneway 字段，确保它是标准的 Boolean 类型
    def parse_bool(x):
        if isinstance(x, str):
            return x.lower() == 'true'
        return bool(x)

    df['oneway'] = df['oneway'].apply(parse_bool)
    print("✅ oneway 字段标准化完成。")

    # 2. 建立索引以加速查找 (u, v)
    existing_edges = set(zip(df['u'], df['v']))

    # 3. 筛选出需要检查的双向道 (oneway = False)
    bidirectional_df = df[df['oneway'] == False].copy()
    
    new_rows = []
    count_already_exist = 0
    count_need_expansion = 0

    print(f"正在检查 {len(bidirectional_df)} 条双向道路的连通性...")

    # 4. 遍历检查与展开
    for index, row in bidirectional_df.iterrows():
        u, v = row['u'], row['v']
        
        # 核心逻辑：检查反向边 (v -> u) 是否已经存在
        if (v, u) in existing_edges:
            count_already_exist += 1
        else:
            count_need_expansion += 1
            # 如果不存在，则创建反向边
            new_row = row.copy()
            new_row['u'] = v
            new_row['v'] = u
            
            # 翻转几何信息 (LineString)
            if 'geometry' in new_row and isinstance(new_row['geometry'], str):
                try:
                    geom = shapely.wkt.loads(new_row['geometry'])
                    if isinstance(geom, LineString):
                        reversed_geom = LineString(list(geom.coords)[::-1])
                        new_row['geometry'] = shapely.wkt.dumps(reversed_geom)
                except Exception:
                    pass
            
            new_rows.append(new_row)

    # 5. 输出统计报告
    print("-" * 40)
    print(f"📊 检查报告 (Step 4: Bidirectional Expansion)")
    print(f"  - 双向道总记录数 (oneway=False): {len(bidirectional_df)}")
    print(f"  - 已存在的反向边 (无需操作):     {count_already_exist}")
    print(f"  - 缺失并新生成的反向边:         {count_need_expansion}")
    
    if count_need_expansion == 0:
        print("\n✅ 结论：数据已经是完全拓扑展开的状态，没有生成新边。")
        print("   (这说明你的 Step 3 数据已经非常干净了)")
    else:
        print(f"\n⚠️ 结论：检测到拓扑缺失，已补全 {count_need_expansion} 条边。")
    print("-" * 40)

    # 6. 合并数据并保存
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        final_df = pd.concat([df, new_df], ignore_index=True)
    else:
        final_df = df

    # 保存
    try:
        final_df.to_csv(output_file, index=False)
        print(f"💾 处理完成！文件已保存为:\n{output_file}")
        print(f"最终数据集行数: {len(final_df)}")
    except Exception as e:
        print(f"❌ 保存失败: {e}")

if __name__ == "__main__":
    process_bidirectional_edges()

正在读取文件: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step3.csv ...


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\1097837975.py:21: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


✅ oneway 字段标准化完成。
正在检查 77198 条双向道路的连通性...
----------------------------------------
📊 检查报告 (Step 4: Bidirectional Expansion)
  - 双向道总记录数 (oneway=False): 77198
  - 已存在的反向边 (无需操作):     77198
  - 缺失并新生成的反向边:         0

✅ 结论：数据已经是完全拓扑展开的状态，没有生成新边。
   (这说明你的 Step 3 数据已经非常干净了)
----------------------------------------
💾 处理完成！文件已保存为:
D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step4.csv
最终数据集行数: 91227


In [19]:
import pandas as pd
import networkx as nx
import os

# ================= 配置区域：绝对路径 =================
# 输入文件 (Step 4 的产出)
input_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step4.csv"
# 输出文件 (Step 5 的产出)
output_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step5.csv"
# ===================================================

def filter_island_nodes():
    print(f"🔄 [Step 5] 正在读取数据: {input_file} ...")
    if not os.path.exists(input_file):
        print(f"❌ 错误: 找不到文件，请检查路径:\n{input_file}")
        return

    # 读取数据，low_memory=False 防止列类型推断警告
    df = pd.read_csv(input_file, low_memory=False)
    original_edge_count = len(df)
    
    # 【关键优化】强制将 u, v 列转换为字符串，防止因 int/str 类型混杂导致匹配失败
    df['u'] = df['u'].astype(str)
    df['v'] = df['v'].astype(str)
    
    # ---------------------------------------------------------
    # 1. 构建有向图 (Construct Directed Graph)
    # ---------------------------------------------------------
    print("🕸️  正在构建有向图 (DiGraph)...")
    # 使用 DiGraph 确保方向性 (A->B 不等于 B->A)
    G = nx.from_pandas_edgelist(df, source='u', target='v', create_using=nx.DiGraph)
    
    original_node_count = G.number_of_nodes()
    print(f"   - 原始网络节点数: {original_node_count}")
    print(f"   - 原始网络边数:   {original_edge_count}")

    # ---------------------------------------------------------
    # 2. 提取最大强连通分量 (Extract Largest SCC)
    # ---------------------------------------------------------
    print("🔍 正在计算强连通分量 (SCC)...")
    # nx.strongly_connected_components 返回的是一个生成器
    # 这里的逻辑是：找出所有“强连通”的子图，然后取节点数最多的那一个
    scc_generator = nx.strongly_connected_components(G)
    largest_scc_nodes = max(scc_generator, key=len)
    
    # 转换为集合，提升查找速度至 O(1)
    valid_nodes_set = set(largest_scc_nodes)
    
    kept_node_count = len(valid_nodes_set)
    removed_node_count = original_node_count - kept_node_count
    
    print("-" * 50)
    print(f"🧩 连通性分析结果 (L-SCC):")
    print(f"   - 最大主连通分量节点数: {kept_node_count}")
    print(f"   - 剔除孤岛/死胡同节点数: {removed_node_count}")
    print(f"   - 节点保留率: {kept_node_count / original_node_count * 100:.2f}%")
    print("-" * 50)

    # ---------------------------------------------------------
    # 3. 反向过滤 DataFrame (Back-Filtering)
    # ---------------------------------------------------------
    print("✂️  正在根据主连通分量执行剪枝...")
    
    # 核心逻辑：保留“起点”和“终点”都在主连通分量里的边
    # 只要有一端在孤岛上，这条边就必须切除，否则后续算最短路会报错
    mask = df['u'].isin(valid_nodes_set) & df['v'].isin(valid_nodes_set)
    df_clean = df[mask].copy()
    
    new_edge_count = len(df_clean)
    removed_edge_count = original_edge_count - new_edge_count

    # ---------------------------------------------------------
    # 4. 保存与验证
    # ---------------------------------------------------------
    print(f"💾 正在保存清洗后的数据...")
    try:
        df_clean.to_csv(output_file, index=False)
        print(f"✅ 保存成功: {output_file}")
    except PermissionError:
        print(f"❌ 保存失败: 文件可能被占用，请关闭 Excel 后重试。")
        return

    print("\n📊 最终统计报告:")
    print(f"   - 最终保留边数: {new_edge_count}")
    print(f"   - 剔除无效边数: {removed_edge_count}")
    print(f"   - 边保留率: {new_edge_count / original_edge_count * 100:.2f}%")

    # ---------------------------------------------------------
    # 5. 合理性自检
    # ---------------------------------------------------------
    retention_rate = new_edge_count / original_edge_count
    print("\n🧐 数据质量评价:")
    if retention_rate > 0.95:
        print("   🟢 [完美] 保留率 > 95%。路网连通性极好，几乎没有断头路。")
    elif retention_rate > 0.85:
        print("   🟢 [优秀] 保留率 > 85%。正常的城市路网，剔除了边缘噪声。")
    elif retention_rate > 0.70:
        print("   🟡 [警告] 保留率 70%-85%。请检查是否误删了重要区域（如因单行道设置错误导致整个区不可达）。")
    else:
        print("   🔴 [严重错误] 保留率 < 70%！路网破碎严重，主干道可能中断，请回溯检查 Step 3 或 Step 4。")

if __name__ == "__main__":
    filter_island_nodes()

🔄 [Step 5] 正在读取数据: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step4.csv ...
🕸️  正在构建有向图 (DiGraph)...
   - 原始网络节点数: 37162
   - 原始网络边数:   91227
🔍 正在计算强连通分量 (SCC)...
--------------------------------------------------
🧩 连通性分析结果 (L-SCC):
   - 最大主连通分量节点数: 36946
   - 剔除孤岛/死胡同节点数: 216
   - 节点保留率: 99.42%
--------------------------------------------------
✂️  正在根据主连通分量执行剪枝...
💾 正在保存清洗后的数据...
✅ 保存成功: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step5.csv

📊 最终统计报告:
   - 最终保留边数: 90924
   - 剔除无效边数: 303
   - 边保留率: 99.67%

🧐 数据质量评价:
   🟢 [完美] 保留率 > 95%。路网连通性极好，几乎没有断头路。


In [20]:
import pandas as pd
import shapely.wkt
import numpy as np
import os

# ================= 配置区域 =================
input_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step5.csv"
output_edges_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step6.csv"
output_nodes_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\nodes_drive_step6.csv"
# ===========================================

def process_attributes_and_coordinates():
    print(f"🔄 [Step 6] 读取 Step 5 数据: {input_file} ...")
    if not os.path.exists(input_file):
        print(f"❌ 错误: 找不到文件 {input_file}")
        return

    # 读取数据，u 和 v 必须读为字符串以防 ID 精度丢失
    df = pd.read_csv(input_file, dtype={'u': str, 'v': str})
    
    # =========================================================
    # 任务 1: 计算通行时间 (Impedance Calculation)
    # =========================================================
    print("⏱️  正在计算通行时间 (travel_time)...")
    
    # 检查除零错误 (虽然 Step 5 检查过，再保险一次)
    if (df['maxspeed'] <= 0).any():
        print("⚠️  警告: 发现 maxspeed <= 0 的记录，将用默认值 1.0 m/s 替换以避免除零错误。")
        df.loc[df['maxspeed'] <= 0, 'maxspeed'] = 1.0

    # 计算时间 (秒)
    df['travel_time'] = df['length'] / df['maxspeed']
    
    # 保留 4 位小数
    df['travel_time'] = df['travel_time'].round(4)
    
    print(f"   - 通行时间计算完成。示例: 长度 {df.iloc[0]['length']}m / 速度 {df.iloc[0]['maxspeed']}m/s = {df.iloc[0]['travel_time']}s")

    # =========================================================
    # 任务 2: 提取并构建节点坐标表 (Node Coordinate Extraction)
    # =========================================================
    print("📍 正在从几何列提取节点坐标 (Node Extraction)...")
    
    # 这里的逻辑是：边的起点坐标 = u 的坐标，边的终点坐标 = v 的坐标
    # 我们用字典来存储，key=node_id, value=(x, y)，利用字典特性自动去重
    node_coords = {}
    
    # 为了效率，我们只解析 geometry 列
    # 预先检查 geometry 是否存在且有效
    valid_geom_mask = df['geometry'].notna()
    
    count = 0
    total = len(df)
    
    # 使用 iterrows 可能会慢，改用 zip 遍历必要列会快很多
    for u, v, wkt in zip(df.loc[valid_geom_mask, 'u'], 
                         df.loc[valid_geom_mask, 'v'], 
                         df.loc[valid_geom_mask, 'geometry']):
        try:
            line = shapely.wkt.loads(wkt)
            if hasattr(line, 'coords'):
                coords = list(line.coords)
                # 起点对应 u
                start_pt = coords[0]
                # 终点对应 v
                end_pt = coords[-1]
                
                # 存入字典 (x=lon, y=lat)
                # 只有当节点不在字典里时才写入（避免重复写入，虽然覆盖也没事）
                if u not in node_coords:
                    node_coords[u] = {'osmid': u, 'x': start_pt[0], 'y': start_pt[1]}
                
                if v not in node_coords:
                    node_coords[v] = {'osmid': v, 'x': end_pt[0], 'y': end_pt[1]}
                    
        except Exception as e:
            # 极少数情况 geometry 解析失败
            pass
            
        count += 1
        if count % 20000 == 0:
            print(f"   - 已处理 {count}/{total} 条边...")

    # 将字典转为 DataFrame
    nodes_df = pd.DataFrame.from_dict(node_coords, orient='index')
    
    print(f"   - 提取完成！共找到 {len(nodes_df)} 个唯一节点。")

    # =========================================================
    # 任务 3: 校验与保存
    # =========================================================
    
    # 1. 保存新的边文件 (带 travel_time)
    print(f"💾 正在保存 Edge 文件: {output_edges_file}")
    df.to_csv(output_edges_file, index=False)
    
    # 2. 保存新的点文件 (用于 KDTree)
    print(f"💾 正在保存 Node 文件: {output_nodes_file}")
    nodes_df.to_csv(output_nodes_file, index=False)
    
    # 3. 完整性检查
    print("-" * 40)
    print("📊 [Step 6 校验报告]")
    
    # 检查是否有 NaN 坐标
    missing_coords = nodes_df[['x', 'y']].isnull().any(axis=1).sum()
    if missing_coords == 0:
        print("   ✅ 坐标完整性: 所有节点均有 (x, y) 坐标。")
    else:
        print(f"   ❌ 坐标缺失: 有 {missing_coords} 个节点无法提取坐标！")
        
    # 检查 travel_time 异常
    neg_time = (df['travel_time'] < 0).sum()
    zero_time = (df['travel_time'] == 0).sum()
    if neg_time == 0 and zero_time == 0:
        print("   ✅ 时间合理性: 所有 travel_time 均为正数。")
    else:
        print(f"   ⚠️ 时间异常: 发现 {neg_time} 个负值，{zero_time} 个零值。")
        
    print("-" * 40)

if __name__ == "__main__":
    process_attributes_and_coordinates()

🔄 [Step 6] 读取 Step 5 数据: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step5.csv ...


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\2167630227.py:19: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, dtype={'u': str, 'v': str})


⏱️  正在计算通行时间 (travel_time)...
   - 通行时间计算完成。示例: 长度 55.48935589611431m / 速度 8.94m/s = 6.2069s
📍 正在从几何列提取节点坐标 (Node Extraction)...
   - 已处理 20000/90924 条边...
   - 已处理 40000/90924 条边...
   - 已处理 60000/90924 条边...
   - 已处理 80000/90924 条边...
   - 提取完成！共找到 36946 个唯一节点。
💾 正在保存 Edge 文件: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step6.csv
💾 正在保存 Node 文件: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\nodes_drive_step6.csv
----------------------------------------
📊 [Step 6 校验报告]
   ✅ 坐标完整性: 所有节点均有 (x, y) 坐标。
   ✅ 时间合理性: 所有 travel_time 均为正数。
----------------------------------------


In [21]:
import pandas as pd
import shapely.wkt
import os
import numpy as np

# ================= 配置区域：绝对路径 =================
input_file = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step5.csv"

# 输出文件
output_edges = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step6.csv"
output_nodes = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\nodes_drive_step6.csv"
# ===================================================

def process_step6():
    print(f"🔄 [Step 6] 读取 Step 5 数据: {input_file} ...")
    if not os.path.exists(input_file):
        print(f"❌ 错误: 找不到文件 {input_file}")
        return

    # 读取数据，u 和 v 读为字符串，防止长数字被截断或精度丢失
    df = pd.read_csv(input_file, dtype={'u': str, 'v': str})
    
    # ---------------------------------------------------------
    # 任务 1: 计算物理阻抗 (travel_time)
    # ---------------------------------------------------------
    print("⏱️  正在计算通行时间 (travel_time)...")
    
    # 防御性编程：检查 maxspeed 是否有 0 或负数（防止除零报错）
    invalid_speed_mask = df['maxspeed'] <= 0
    if invalid_speed_mask.any():
        print(f"⚠️  发现 {invalid_speed_mask.sum()} 条记录速度异常 (<=0)，已修正为 1.0 m/s。")
        df.loc[invalid_speed_mask, 'maxspeed'] = 1.0

    # 核心公式：时间 = 距离 / 速度
    df['travel_time'] = df['length'] / df['maxspeed']
    
    # 精度处理：保留 4 位小数（例如 0.1234 秒）
    df['travel_time'] = df['travel_time'].round(4)
    
    # 再次检查计算后的时间是否有 0 值 (可能因为 length 极短)
    # 如果时间为 0，这在图算法里会导致问题，强制设为 0.001 秒
    zero_time_mask = df['travel_time'] == 0
    if zero_time_mask.any():
        print(f"⚠️  发现 {zero_time_mask.sum()} 条记录计算时间为 0，已修正为 0.001s。")
        df.loc[zero_time_mask, 'travel_time'] = 0.001

    # ---------------------------------------------------------
    # 任务 2: 从几何列提取节点坐标
    # ---------------------------------------------------------
    print("📍 正在解析 geometry 提取节点坐标...")
    
    node_coords = {} # 字典结构: {node_id: {'osmid': id, 'x': lon, 'y': lat}}
    
    # 使用 zip 高效遍历
    # 逻辑：对于每一条边，起点坐标属于 u，终点坐标属于 v
    for u, v, wkt_str in zip(df['u'], df['v'], df['geometry']):
        try:
            # 解析 WKT 字符串
            geom = shapely.wkt.loads(wkt_str)
            coords = list(geom.coords)
            
            # 提取首尾点
            start_pt = coords[0]  # (x, y)
            end_pt = coords[-1]   # (x, y)
            
            # 写入字典（自动去重）
            if u not in node_coords:
                node_coords[u] = {'osmid': u, 'x': start_pt[0], 'y': start_pt[1]}
            
            if v not in node_coords:
                node_coords[v] = {'osmid': v, 'x': end_pt[0], 'y': end_pt[1]}
                
        except Exception as e:
            # 仅在几何解析严重错误时触发，一般不会发生
            pass

    # 转换为 DataFrame
    nodes_df = pd.DataFrame.from_dict(node_coords, orient='index')
    
    # ---------------------------------------------------------
    # 任务 3: 深度自检 (Deep Self-Check)
    # ---------------------------------------------------------
    print("\n🧐 [数据质量自检报告]")
    print("-" * 40)
    
    # --- 检查 1: 节点完整性 ---
    # 边表里出现过的所有点，必须都有坐标
    edges_nodes = set(df['u']).union(set(df['v']))
    extracted_nodes = set(nodes_df['osmid'])
    missing_nodes = edges_nodes - extracted_nodes
    
    if len(missing_nodes) == 0:
        print("   ✅ [完整性] 节点提取完美匹配 (边表中的每个点都找到了坐标)。")
    else:
        print(f"   ❌ [错误] 有 {len(missing_nodes)} 个节点缺失坐标！这会导致后续建图失败。")
        # 如果有缺失，通常是因为 geometry 为空或解析失败，需要检查源数据
    
    # --- 检查 2: 时间合理性 ---
    min_time = df['travel_time'].min()
    max_time = df['travel_time'].max()
    mean_time = df['travel_time'].mean()
    
    print(f"   ✅ [属性] 通行时间范围: {min_time}s ~ {max_time}s (均值: {mean_time:.2f}s)")
    if max_time > 3600:
        print("      ⚠️  注意：存在通行时间 > 1小时的边，请确认这是否为超长高速路段。")
    
    # --- 检查 3: 坐标有效性 (Geo Validity) ---
    # 检查经纬度是否在地球范围内
    # x (经度): -180 ~ 180, y (纬度): -90 ~ 90
    valid_geo = (
        (nodes_df['x'] >= -180) & (nodes_df['x'] <= 180) &
        (nodes_df['y'] >= -90) & (nodes_df['y'] <= 90)
    )
    if valid_geo.all():
        print(f"   ✅ [坐标] 所有 {len(nodes_df)} 个节点的经纬度均在合法范围内。")
        print(f"      示例坐标: ID {nodes_df.iloc[0]['osmid']} -> ({nodes_df.iloc[0]['x']:.5f}, {nodes_df.iloc[0]['y']:.5f})")
    else:
        invalid_count = (~valid_geo).sum()
        print(f"   ❌ [坐标] 发现 {invalid_count} 个节点坐标异常（超出地球范围）！")

    print("-" * 40)

    # ---------------------------------------------------------
    # 任务 4: 保存
    # ---------------------------------------------------------
    print(f"💾 正在保存 Edge 文件 (含 travel_time): {output_edges}")
    df.to_csv(output_edges, index=False)
    
    print(f"💾 正在保存 Node 文件 (含 x, y): {output_nodes}")
    nodes_df.to_csv(output_nodes, index=False)
    
    print("\n✅ Step 6 处理全部完成。")

if __name__ == "__main__":
    process_step6()

🔄 [Step 6] 读取 Step 5 数据: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step5.csv ...


C:\Users\Kam1\AppData\Local\Temp\ipykernel_29108\2553082860.py:21: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, dtype={'u': str, 'v': str})


⏱️  正在计算通行时间 (travel_time)...
📍 正在解析 geometry 提取节点坐标...

🧐 [数据质量自检报告]
----------------------------------------
   ✅ [完整性] 节点提取完美匹配 (边表中的每个点都找到了坐标)。
   ✅ [属性] 通行时间范围: 0.0967s ~ 509.9792s (均值: 15.21s)
   ✅ [坐标] 所有 36946 个节点的经纬度均在合法范围内。
      示例坐标: ID 37293968 -> (-76.76243, 39.27487)
----------------------------------------
💾 正在保存 Edge 文件 (含 travel_time): D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_drive_step6.csv
💾 正在保存 Node 文件 (含 x, y): D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\nodes_drive_step6.csv

✅ Step 6 处理全部完成。
